# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\Y_Han\OneDrive\Udacity\Data Engineering Nanodegree\2. DataModeling\2.5 - Project, Data Modeling with Apache Cassandra\yougun, rev02 - Polished\Data-Modeling-with-Apache-Cassandra


In [7]:
pd.read_csv(file_path_list[0])

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,NaN,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540920e+12,38,NaN,200,1.541110e+12,39
1,NaN,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540340e+12,139,NaN,200,1.541110e+12,8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,You Gotta Be,200,1.541110e+12,8
3,NaN,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540340e+12,139,NaN,200,1.541110e+12,8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Flat 55,200,1.541110e+12,8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Quem Quiser Encontrar O Amor,200,1.541110e+12,8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Eriatarka,200,1.541110e+12,8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Becoming Insane,200,1.541110e+12,8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Congratulations,200,1.541110e+12,8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540340e+12,139,Once again,200,1.541110e+12,8


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skip the header
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' :'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exeption as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Main task 1: Queries to return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
##### Table Design
The primary key is made of 
- Partition key : sessionId
- Clustering key : itemInSession

This is done to accommodate the SELECT queries by sessionId and itemInSession

In [8]:
### This cell does three things
## 1. Create the table, "music_library_by_session_sessionitem"
## 2. Insert the data into the table, "music_library_by_session_sessionitem"
## 3. Verify the table with an exmple SELECT query


## part 1. Create a table

# Define table name
table = "music_library_by_session_sessionitem"

# drop an existing table if any
try:
    query = "DROP TABLE IF EXISTS " + table
    session.execute(query)
except Exception as e:
    print(e) 

# create a table
query = "CREATE TABLE IF NOT EXISTS " + table
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)             
    
    
    
## part 2. Insert data into the table

# Load data from "event_datafile_new.csv"
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip the header
    next(csvreader) 
    # insert the data line by line
    for line in csvreader:
        query = "INSERT INTO music_library_by_session_sessionitem (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        
        
## part 3. Verify the table with an exmple SELECT query

# Example SELECT query: Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4
query = "SELECT artist, song_title, song_length FROM music_library_by_session_sessionitem WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
# Example SELECT query: Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4
query = "SELECT artist, song_title, song_length FROM music_library_by_session_sessionitem WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
        
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Main task 2: Queries to return the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
The primary key is made of 
- Partition key : userid, sessionid
- Clustering key : itemInSession

Userid and session id are used as a composite partition key to partiion data by userid and sessionid and increase the SELECT querry performance.
itemInSession is used as clustering key to return the output sorted by itemInSession.

In [9]:
### This cell does three things
## 1. Create the table, "music_library_by_user_session_sessionitem"
## 2. Insert the data into the table, "music_library_by_user_session_sessionitem"
## 3. Verify the table with an exmple SELECT query



## part 1. Create a table

# Define table name
table = "music_library_by_user_session_sessionitem"

# drop an existing table if any
try:
    query = "DROP TABLE IF EXISTS " + table
    session.execute(query)
except Exception as e:
    print(e) 

# create a table
query = "CREATE TABLE IF NOT EXISTS " + table
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, user_first_name text, user_last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)       

    
    
## part 2. Insert data into the table

# Load data from "event_datafile_new.csv"
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader) 
    # insert the data line by line    
    for line in csvreader:
        query = "INSERT INTO music_library_by_user_session_sessionitem (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s ,%s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

        
        
## part 3. Verify the table with an exmple SELECT query

# Example SELECT query: Return only name of artist, song and user's first and last name sorted by itemInSession for userid =10, session id =182
query = "SELECT artist, song_title, user_first_name, user_first_name, user_last_name \
        FROM music_library_by_user_session_sessionitem WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Main task 3: Queries to return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
The primary key is made of 
- Partition key : song
- Clustering key : userId

Song is used as Partition key so that we can query by song in SELECT statement. The userId is used as clustering key to identify user uniquely in addition to the partition key (i.e. song)

In [10]:
### This cell does three things
## 1. Create the table, "music_library_by_song"
## 2. Insert the data into the table, "music_library_by_song"
## 3. Verify the table with an exmple SELECT query


## part 1. Create a table

# Define table name
table = "music_library_by_song"

# drop an existing table if any
try:
    query = "DROP TABLE IF EXISTS " + table
    session.execute(query)
except Exception as e:
    print(e) 
    
# create a table
query = "CREATE TABLE IF NOT EXISTS " + table
query = query + "(song_title text, user_id int, user_last_name text, user_first_name text, PRIMARY KEY(song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)       



## part 2. Insert data into the table

# Load data from "event_datafile_new.csv"
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    # insert data line by line
    for line in csvreader:
        query = "INSERT INTO music_library_by_song (song_title, user_id, user_last_name, user_first_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[4], line[1]))

## part 3. Verify the table with an exmple SELECT query

# Example SELECT query: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT user_last_name, user_first_name FROM music_library_by_song WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_last_name, row.user_first_name)

KeyboardInterrupt: 

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
tables = ["music_library_by_session_sessionitem","music_library_by_user_session_sessionitem","music_library_by_song"]

for table in tables:
    try:
        query = "DROP TABLE IF EXISTS " + table
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()